In [1]:
from langgraph_sdk import get_client

In [2]:
client = get_client(url="http://localhost:8123")

In [9]:
#topic = "Top 5 chip providers for LLM Training"
topic = "Fact checking the Trump's claim: people are eating the dogs and cats in Springfield"

In [13]:
schema = {
    "type": "object",
    "properties": {
        "references": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "title": {"type": "string", "description": "title name of the article"},
                    "url": {
                        "type": "string",
                        "description": "url link to the news article",
                    },
                    "news_source_trustworthy_score": {
                        "type": "number",
                        "description": "0 to 1 score in terms of whether the news source is trustworthy in general",
                    },
                    "support_claim": {
                        "type": "boolean",
                        "description": "boolean value to indicate whether the news article support the claim",
                    },
                },
                "required": ["title", "url", "news_source_trustworthy_score", "support_claim"],
            },
            "description": "List of companies",
        },
        "factualness_score": {
            "type": "number",
            "description": "0 to 1 score in terms of factualness based on the references from research above"
        }
    },
    "required": ["references"],
}

In [11]:
from langchain_core.messages import convert_to_messages

# Thread
thread = await client.threads.create()

# Model
#llm = "anthropic/claude-3-5-sonnet-20240620"
llm = "openai/gpt-4o"

# Stream
async for event in client.runs.stream(
    thread["thread_id"],
    assistant_id="agent",
    input={
        "topic": topic,
        "extraction_schema": schema,
        "configurable": {"model_name": llm},
    },
    stream_mode="values",
):
    messages = event.data.get("messages", None)
    if messages:
        print(convert_to_messages(messages)[-1])
    print("=" * 25)

content='' additional_kwargs={'additional_kwargs': {'tool_calls': [{'id': 'call_tu04dPECbqjZv2G36M6Nknjb', 'function': {'arguments': '{"query":"Trump claim people eating dogs cats Springfield fact check"}', 'name': 'search'}, 'type': 'function', 'index': 0}], 'refusal': None}, 'response_metadata': {'token_usage': {'completion_tokens': 19, 'prompt_tokens': 618, 'total_tokens': 637, 'completion_tokens_details': None}, 'model_name': 'openai/gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1bb46167f9', 'finish_reason': 'tool_calls', 'logprobs': None}, 'example': False, 'invalid_tool_calls': [], 'usage_metadata': {'input_tokens': 618, 'output_tokens': 19, 'total_tokens': 637}} response_metadata={} id='run-3d22a261-6e64-4ace-b768-75722ba68ca3-0' tool_calls=[{'name': 'search', 'args': {'query': 'Trump claim people eating dogs cats Springfield fact check'}, 'id': 'call_tu04dPECbqjZv2G36M6Nknjb', 'type': 'tool_call'}]
content='[{"url": "https://www.newsweek.com/donald-trump-cats-dogs-claim-de

In [14]:
current_state = await client.threads.get_state(thread["thread_id"])

In [15]:
from IPython.display import Markdown, display


# def format_llm_chip_info(data):
#     markdown_text = "# Top 5 Chip Providers for LLM Training\n\n"

#     for company in data["companies"]:
#         markdown_text += f"""
# ## {company['name']}

# **Key Technologies:** {company['technologies']}

# **Market Share:** {company['market_share']}

# **Key Powers:** {company.get('key_powers', 'Not specified')}

# **Future Outlook:** {company['future_outlook']}

# ---
# """

#     return Markdown(markdown_text)


# # Display the formatted markdown
# display(format_llm_chip_info(current_state["values"]["info"]))

In [16]:
current_state["values"]["info"]

{'references': [{'url': 'https://news.yahoo.com/news/truth-springfield-ohio-immigrants-eating-182913769.html',
   'title': 'The truth about Springfield, Ohio: Are immigrants eating dogs as Trump says? What to know',
   'support_claim': False,
   'news_source_trustworthy_score': 0.8},
  {'url': 'https://www.economist.com/the-economist-explains/2024/09/11/donald-trump-says-immigrants-are-eating-springfields-pets-what',
   'title': 'Donald Trump says immigrants are eating Springfield’s pets. What?',
   'support_claim': False,
   'news_source_trustworthy_score': 0.9},
  {'url': 'https://www.dispatch.com/story/news/2024/09/11/truth-fact-check-people-eating-dogs-springfield-ohio-trump-vance-harris-debate/75171964007/',
   'title': 'Fact check: Are people eating dogs in Springfield, Ohio, as Trump says?',
   'support_claim': False,
   'news_source_trustworthy_score': 0.7},
  {'url': 'https://www.usatoday.com/story/news/politics/elections/2024/09/11/canton-woman-accused-of-eating-cat-not-haiti